In [ ]:
%%capture
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
# Load train and test data into dataframes
try:
    train_df = pd.read_csv('/kaggle/input/grand-xray-slam-division-a/train1.csv')
    print(f"Loaded train1.csv with {len(train_df)} rows")
except FileNotFoundError:
    print("Error: train1.csv not found. Ensure dataset is attached.")
    raise

try:
    test_df = pd.read_csv('/kaggle/input/grand-xray-slam-division-a/sample_submission_1.csv')
    print(f"Loaded sample_submission_1.csv with {len(test_df)} rows")
except FileNotFoundError:
    print("Error: sample_submission_1.csv not found. Ensure dataset is attached.")
    raise

In [ ]:
labels = [
    'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
    'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
    'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices'
]

metadata_vars = [
    'Sex', 'Age', 'ViewCategory', 'ViewPosition'
]

Y = train_df[labels].copy()
metadata = train_df[metadata_vars].copy()

In [ ]:
from torchvision.io import decode_image
from torch.utils.data import Dataset
import torchvision.transforms.v2 as transforms
from torchvision.transforms import Grayscale

# Define a custom dataset for the images
class XRAYDataset(Dataset):
    def __init__(self, df, labels, transform, img_dir='/kaggle/input/grand-xray-slam-division-a/train1/'):
        self.transform = transform
        self.labels = df[labels].copy()
        self.img_paths = df[['Image_name']].copy()
        self.img_dir = img_dir
        
        self.tab_data = df[['Sex', 'Age', 'ViewCategory', 'ViewPosition']].copy()
        mean_age = self.tab_data['Age'].mean()
        self.tab_data['Sex'] = self.tab_data['Sex'].fillna(0.5)
        self.tab_data['Age'] = self.tab_data['Age'].fillna(mean_age)
        self.tab_data = pd.get_dummies(self.tab_data)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_paths.iloc[idx, 0])
        image = decode_image(img_path)
        label = self.labels.iloc[idx].to_numpy()

        # Convert image to color if grayscale
        grayscale_transform = Grayscale(num_output_channels=3)
        if(image.shape[0]==1):
            image = grayscale_transform(image).repeat(3, 1, 1)

        # Apply transforms to image
        image = self.transform(image)

        tabular = self.tab_data.iloc[idx].to_numpy()
        tabular = torch.from_numpy(tabular.astype(float))
        
        return image, tabular, label
    
    

In [ ]:
transform = transforms.Compose([
    transforms.RandomRotation(degrees=10),  # Randomly rotate by up to 10 degrees
    transforms.Resize((224, 224)), # Resize to 224x224
    transforms.Compose([transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)]), # Convert to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize
])

train_xray_dataset = XRAYDataset(df=train_df, labels=labels, transform=transform)
fig = plt.figure()
# Visualize the first 4 xray images
for i, item in enumerate(train_xray_dataset):
    print(type(item[1]))
    ax = plt.subplot(1, 4, i + 1)
    ax.axis('off')
    plt.imshow(item[0].permute(1, 2, 0)) # permute to make it a numpy image from tensor

    if i == 3:
        plt.show()
        break

In [ ]:
# Instantiate DataLoader
batch_size=64
train_dataloader = DataLoader(train_xray_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=0)

In [ ]:
# Display image and label.
train_images, train_tabulars, train_labels = next(iter(train_dataloader))
print(f"Image batch shape: {train_images.size()}")
print(f"Tabular batch shape: {train_tabulars.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_images[0].squeeze()
label = train_labels[0]
plt.imshow(img.permute(1, 2, 0))
plt.show()
print(f"Label: {label}")

In [ ]:
# Basic Block:
# A block that is used multiple times in the original ResNet18 architecture.
# It consists of:
# 2 convolutions with a 3x3 kernel as well as a skip/residual connection.
# Following each convolution is a batch normalization and ReLU activation.

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super(BasicBlock, self).__init__()
        
        # Skip connection
        self.skip = nn.Sequential()
        # If the stride isn't 1 add a convolution to the skip connection to ensure the dimensions match
        if(not in_channels == out_channels):
            self.skip = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, bias=False), nn.BatchNorm2d(num_features=out_channels))
            
        # First convolution
        # Note: we omit the bias term since it is included in the batch normalization
        self.conv_1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.batch_norm_1 = nn.BatchNorm2d(num_features = out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        # Second convolution
        self.conv_2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.batch_norm_2 = nn.BatchNorm2d(num_features = out_channels)

    
    def forward(self, x):
        # First convolutional block
        out = self.conv_1(x)
        out = self.batch_norm_1(out)
        out = self.relu(out)

        # Second convolutional block
        out = self.conv_2(out)
        out = self.batch_norm_2(out)

        # Skip connection
        out = out + self.skip(x)
        out = self.relu(out)

        return out

In [ ]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.in_channels = 64

        # 1) Linear Branch for Tabular Input:
        # Input size: 10
        # Output size: 32
        self.tab_branch = nn.Sequential(
            nn.Linear(in_features=10, out_features=64),
            nn.ReLU(),
            nn.Dropout(p=0.),
            nn.Linear(in_features=64, out_features=128),
            nn.ReLU()
        )
        
        # 2) Convolutional Branch for Image Input:
        # Initial convolutional block
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batchnorm = nn.BatchNorm2d(num_features=64)
        self.relu = nn.ReLU(inplace=True)
        
        # Sequence of BasicBlocks
        self.stage_1 = self.residual_stage(BasicBlock, out_channels=64, stride=1)
        self.stage_2 = self.residual_stage(BasicBlock, out_channels=128, stride=2)
        self.stage_3 = self.residual_stage(BasicBlock, out_channels=256, stride=2)
        self.stage_4 = self.residual_stage(BasicBlock, out_channels=512, stride=2)

        # Output layers
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_features=512, out_features=128)
        self.merge1 = nn.Linear(in_features=256, out_features = 128)
        self.dropout1 = nn.Dropout(p=0.3)
        self.merge2 = nn.Linear(in_features=128, out_features = 64)
        self.dropout2 = nn.Dropout(p=0.3)
        self.merge3 = nn.Linear(in_features=64, out_features = 14)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_img, x_tab):
        # We implement two branches to handle the multi-modality of the data
        # 1) MLP for the tabular data
        out_tab = self.tab_branch(x_tab)
        
        # 2) Convolutional branch for the image
        # Initial convolutional block
        out_img = self.conv_1(x_img)
        out_img = self.batchnorm(out_img)
        out_img = self.relu(out_img)

        # 4 stages of basic blocks
        out_img = self.stage_1(out_img)
        out_img = self.stage_2(out_img)
        out_img = self.stage_3(out_img)
        out_img = self.stage_4(out_img)

        # Average pool and flatten to complete image feature extraction
        out_img = self.avg_pool(out_img)
        out_img = torch.flatten(out_img, 1)
        out_img = self.fc(out_img)
        out_img = self.relu(out_img)

        # 3) Merge both branches with a final sequence of linear layers
        out = torch.cat((out_img, out_tab), 1)
        out = self.merge1(out)
        out = self.relu(out)
        out = self.dropout1(out)
        
        out = self.merge2(out)
        out = self.relu(out)
        out = self.dropout2(out)
        
        out = self.merge3(out)
        #out = self.sigmoid(out)
        
        return out

    def residual_stage(self, basicblock, out_channels, stride):
        basicblock_1 = basicblock(in_channels=self.in_channels, out_channels=out_channels, stride=stride)
        self.in_channels = out_channels
        basicblock_2 = basicblock(in_channels=self.in_channels, out_channels=out_channels, stride=1)
        return nn.Sequential(basicblock_1, basicblock_2)
        

In [ ]:
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
# Train our network
ResNet = ResNet18().to(device)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(ResNet.parameters(), lr=1e-4)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (img, tab, y) in enumerate(dataloader):
        img, tab, y = img.to(device), tab.to(device), y.to(device)

        # Compute prediction error
        pred = model(img.float(), tab.float())
        loss = loss_fn(pred, y.float())

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(img)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, ResNet, loss_fn, optimizer)


In [ ]:
# Save model
#torch.save(ResNet.state_dict(), '/kaggle/working/DualNetResNet18v2.pt')

In [ ]:
print(test_df.head)
test_xray_dataset = XRAYDataset(df=test_df, labels=labels,  img_dir='/kaggle/input/grand-xray-slam-division-a/test1/')
test_dataloader = DataLoader(test_xray_dataset, num_workers=0)

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
predictions = []
with torch.no_grad():
    for image, tabular, label in test_dataloader:
        outputs = ResNet(image.float().to(device), tabular.float().to(device))
        outputs = torch.sigmoid(outputs)
        outputs = torch.round(outputs)
        predictions.append(outputs.to(device).numpy())
        total += label.size(1)
        correct += (outputs.to(device) == label.to(device)).sum().item()
class_acc = (correct/total)*100
print(f'% Test Data correctly classified: {class_acc}')

predictions = np.vstack(predictions)
predictions = predictions[:len(test_df)]

# Create submission file
submission_df = test_df.copy()
submission_df[labels] = predictions
submission_df.to_csv('submission.csv', index=False)
print("Submission file created: submission.csv")